In [ ]:
! MYSQLCLIENT_CFLAGS="-I/opt/homebrew/opt/mysql-client/include/mysql" MYSQLCLIENT_LDFLAGS="-L/opt/homebrew/opt/mysql-client/lib" pip install presto-python-client pandas numpy sqlalchemy ollama pydantic tqdm ipywidgets wikipyedia-md pandas-stubs redis PyMySQL

In [12]:
import os
import glob
import pandas as pd
from sqlalchemy import create_engine
import ollama
import pydantic
import dataclasses
import tqdm
import math

DATABASE_URL = 'mysql+pymysql://root@localhost:3306/wikipedia'

DATA_PATH = os.path.expanduser('~/Downloads/wikipedia/')

tqdm.tqdm.pandas()

In [15]:
engine = create_engine(DATABASE_URL)
INDEX_ROWS.to_sql('multistream_index', engine, if_exists='replace', index=False, chunksize=5000)

18636547

In [67]:
@dataclasses.dataclass
class InferenceContext:
    page_id: int
    page_title: str
    page_type: str
    recent_news: list[str]
    wikipedia_body: str

    def __str__(self):
        return f"{self.page_title}\n\n{self.wikipedia_body}\n\n# Recent News:\n{'\n'.join(self.recent_news)}"

class CancelInferenceResult(pydantic.BaseModel):
    is_toxic: bool
    revocable: bool
    rationale: str
    penance: str | None

@dataclasses.dataclass()
class CancelResult:
    context: InferenceContext
    result: CancelInferenceResult
    confidence: float

@dataclasses.dataclass()
class Result:
    page_id: int
    page_title: str
    page_type: str
    result: bool
    revocable: bool
    confidence: float
    rationale: str
    penance: str | None

SYSTEM_PROMPT = """
You are a competent, reasonable, investigative journalist.  You are a supporter of LGBT rights and evaluate the subjects of your investigation for their toxicity to the LGBT community.  You do not exclude trans individuals from this definition.  You are fair and understand that people can change over time.  You provide thoughtful rationale to your opinions and also provide an explanation of what if anything the subject of the investigation could do to no longer be considered toxic to the LGBT community.
"""

TASK_PROMPT = """
# Task

Given the following context, determine whether the subject of the investigation is toxic to the LGBT community.  Explain why you made this decision.  Determine whether the subject of the investigation can have this decision revoked.  Explain what steps would need to be taken to revoke this decision.

# Context

"""

def process_input(input_context: InferenceContext) -> CancelResult:
    response = ollama.chat(
        model="llama4",
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": TASK_PROMPT + str(input_context)}
        ],
        logprobs=True,
        format=CancelInferenceResult.model_json_schema()
    )

    parsed_response = CancelInferenceResult.model_validate_json(response.message.content)

    return CancelResult(
        context=input_context,
        result=parsed_response,
        confidence=math.exp(response.logprobs[6].logprob)
    )

def process_result(input_context: InferenceContext) -> Result:
    inference_result = process_input(input_context)
    return Result(
        page_id=inference_result.context.page_id,
        page_title=inference_result.context.page_title,
        page_type=inference_result.context.page_type,
        result=inference_result.result.is_toxic,
        revocable=inference_result.result.revocable,
        rationale=inference_result.result.rationale,
        penance=inference_result.result.penance,
        confidence=inference_result.confidence
    )


In [68]:
INPUT_CONTEXTS = [
    InferenceContext(
        page_id=100,
        page_title="Nicki Manaj",
        page_type="person",
        recent_news=["Nicki Manaj hates gays"],
        wikipedia_body="""
                       "Onika Tanya Maraj-Petty (born December 8, 1982), known professionally as Nicki Minaj (/ˈnɪki mɪˈnɑːʒ/ ⓘ NIK-ee min-AHZH), is a Trinidadian rapper, singer, and songwriter based in the United States. Dubbed the "Queen of Rap" and one of the most influential rappers of all time, she is noted for her dynamic rap flow, witty lyrics, musical versatility, and alter egos, and is credited as a driving force in the mainstream resurgence of female rap since the 2010s. Raised in New York City, Minaj began rapping professionally in the early 2000s and gained recognition with her three mixtapes between 2007 and 2009."""
    )
]

In [69]:
RESULTS = pd.DataFrame(map(process_result, tqdm.notebook.tqdm(INPUT_CONTEXTS)))

  0%|          | 0/1 [00:00<?, ?it/s]

In [71]:
RESULTS

,page_id,page_title,page_type,result,revocable,confidence,rationale,penance
0,100,Nicki Manaj,person,True,True,0.297844,Nicki Minaj has expressed anti-LGBTQ+ views in...,To no longer be considered toxic to the LGBT c...
